In [1]:
import sources.logins as lg
from time import sleep
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
warnings.filterwarnings('ignore')

def login_exfm(driver_on = True):
    # generate key using fernet
    # key = Fernet.generate_key().decode()
    key = 'cLAiAdZU1U0sxWWK8sxhF0IWIBP4KrR3xhdu3x1EDQI='
    
    
    # login ExFM and download latest incompleted data
    name,pw = lg.save_id('exfm.txt',key)
    driver = lg.login_url('exfm',name,pw)
    
    
    while True:
        try:
            driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() # select RMA Search	
            break
        except Exception as e:
            print ('Wrong Username or Password. Try again...')
            name,pw = lg.save_id('exfm.txt',key)
            driver = lg.login_url('exfm',name,pw)
    
    d_type_menu = [
                    'Incompleted',
                    'History',
                    'Equipments',
                    'Customers',
                    'No Download',
                    
                    ]
    #border table
    print(f'\n{"_"*50}')
    print(f'{"|  No.|  Function": <49}|')
    print(f'|{"_"*48}|')
    for i in range(1,1+len(d_type_menu)):
        print(f'|{i: >3}  |  {d_type_menu[i-1]: <40}|')
    print(f'|{"_"*48}|') #bottom border
    while True:
        ind = str(input('Select Dataset to download: '))
        try:
            ind = int(ind)
        except:
            if ind.upper() == 'Q' or ind.upper() == 'QUIT':
                break
            else:
                print('Only accept number')
                continue
        if 0<ind<=len(d_type_menu):
            d_type = d_type_menu[ind-1]
            break
        else:
            print(f'Input number must be less than {len(d_type_menu)}\n')
    
    print(d_type)
    
    # driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() # select RMA Search
    
    if d_type == d_type_menu[0]: # Incompleted
        driver.find_element_by_xpath('//*[@id="sidEXPORT_CONSOLIDATED_BUTTON_IMAGE"]').click() #click download
    
    if d_type == d_type_menu[1]: # History
        Select(driver.find_element_by_xpath('//*[@id="sidIN_REPAIR_STATUS"]')).select_by_index(0)
        print('Select Status "All" and download.')
        driver.find_element_by_xpath('//*[@id="sidEXPORT_CONSOLIDATED_BUTTON_IMAGE"]').click() #click download
    
    if d_type == d_type_menu[2]: # Equipments
        driver.find_element_by_xpath('//*[@id="EQP_MGT_LINK"]').click() # Equipments
        sleep(0.5)
        driver.find_element_by_xpath('//*[@id="sidEXPORT_BUTTON_IMAGE"]').click()
    
    if d_type == d_type_menu[3]: # Customers
        driver.find_element_by_xpath('//*[@id="ACC_MGT_LINK"]').click() 
        sleep(0.5)
        driver.find_element_by_xpath('//*[@id="sidEXPORT_BUTTON_IMAGE"]').click()
    

    # check latest file
    i_wait = 0
    while True:
        try: # check file already
            file, ctime = lg.file_latest(folder_name='Downloads')
            if file.endswith('xls'):
                print (f'\nDonwload file {file} succesful at {ctime}')
                break
            else:
                print(f'Please wait for compressing file {file}...{i_wait}s')
                i_wait +=3
                sleep(3)
        except Exception as e: #waiting for download
            if d_type == d_type_menu[-1]: break
            print(f'Please wait for dowloading...{i_wait}s')
            i_wait +=3
            sleep(3)
    if driver_on:
        return driver
    else:
        #finish close driver
        driver.close()
    sleep(2)

def auto_accept(driver):
    WebDriverWait(driver, 5).until(EC.alert_is_present())
    driver.switch_to.alert.accept()

In [2]:
driver = login_exfm()


Select account:
1   |   thongle
2   |   Nguyen
3   |   FFAP0104
0   |   CREATE NEW ACCOUNT
Select account by number: 
Chi duoc nhap so.
Select account by number: 1
thongle is selected to login
Removed Downloads


Create new folder: Downloads
Select Chrome Driver and press enter for authorize Certificate...

Typing your ID...
Typing your password...
Login sucessful!


__________________________________________________
|  No.|  Function                                |
|________________________________________________|
|  1  |  Incompleted                             |
|  2  |  History                                 |
|  3  |  Equipments                              |
|  4  |  Customers                               |
|  5  |  No Download                             |
|________________________________________________|
Select Dataset to download: 5
No Download


#### Clean data customer

In [ ]:
from sqlite3 import connect
import pandas as pd
conn = connect('dealer.db')
file_name = 'files/ACC_TBL_EXPORT_230912.xlsx'
dealers = pd.read_excel(file_name)
dealers.to_sql('dealers',conn,index=False,if_exists='replace')

In [ ]:
q = '''
        SELECT * 
        
        FROM dealers
        WHERE territory1 = 'ETC'
    '''
north = pd.read_sql(q,conn)
north

In [ ]:
from time import sleep

In [ ]:
cus_no_done=[]

In [ ]:
#click customer tab

driver.find_element_by_xpath('//*[@id="ACC_MGT_LINK"]').click()
for cus_no in north['No.']:
    if cus_no in cus_no_done:
        print(f' \nPassed {cus_no}')
    else:
        customer = dealers[dealers['No.']==cus_no]
        customer = customer.reset_index()
    #     display(customer)
        #search customer no.
        driver.find_element_by_xpath('//*[@id="sidIN_ACC_CODE"]').clear()
        driver.find_element_by_xpath('//*[@id="sidIN_ACC_CODE"]').send_keys(cus_no)

        #click search Button
        driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()

        # select first row
        driver.find_element_by_xpath('//*[@id="g[0].sidDISP_CODE_field"]').click()
        sleep(0.5)

        # edit by file
        name = driver.find_element_by_xpath('//*[@id="sidNAME"]')
        name.clear()
        name.send_keys(customer['Pro Name'])
        print(cus_no,customer['Pro Name'])
        territory = driver.find_element_by_xpath('//*[@id="sidTEXT_TERRITORY_NAME"]')
        territory.clear()
        try:
            territory.send_keys(customer['Territory1'])
        except Exception as e:
            print(e)
        area = driver.find_element_by_xpath('//*[@id="sidTEXT_AREA_NAME"]')
        area.clear()
        try:
            area.send_keys(customer['Area1'])
        except Exception as e:
            print(e)

        city = driver.find_element_by_xpath('//*[@id="sidCITY"]')
        city.clear()
        try:
            city.send_keys(customer['City1'])
        except Exception as e:
             print(e)
        
        #clear email
        driver.find_element_by_xpath('//*[@id="sidCONTACT1_PERSON_MAIL"]').clear()
        driver.find_element_by_xpath('//*[@id="sidCONTACT2_PERSON_MAIL"]').clear()
        driver.find_element_by_xpath('//*[@id="sidCONTACT3_PERSON_MAIL"]').clear()
        
        
        # click save
        driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
        sleep(0.5)
        #click return
        driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()
        
        sleep(0.5)
        cus_no_done.append(cus_no)
        
        
print('Done')

In [ ]:
cus_no_done

In [ ]:
# edit by file
name = driver.find_element_by_xpath('//*[@id="sidNAME"]')
name.clear()
name.send_keys(customer['Pro Name'])

territory = driver.find_element_by_xpath('//*[@id="sidTEXT_TERRITORY_NAME"]')
territory.clear()
try:
    territory.send_keys(customer['Territory1'])
except Exception as e:
    print(e)
area = driver.find_element_by_xpath('//*[@id="sidTEXT_AREA_NAME"]')
area.clear()
try:
    area.send_keys(customer['Area1'])
except Exception as e:
    print(e)

city = driver.find_element_by_xpath('//*[@id="sidCITY"]')
city.clear()
try:
    city.send_keys(customer['City1'])
except Exception as e:
     print(e)

In [ ]:
# click save
driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
sleep(0.5)
#click return
driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()
sleep(0.5)

### Installation

In [ ]:
def create_eqp(model,sn,date_installed):
    #create EQP
    driver.find_element_by_xpath('//*[@id="sidADD_BUTTON_IMAGE_2"]').click()
    #search sn

    driver.find_element_by_xpath('//*[@id="sidBODY_NO"]').send_keys(sn)
    driver.find_element_by_xpath('//*[@id="sidBODY_NO"]').send_keys(Keys.RETURN)
    #click model
    driver.find_element_by_xpath('//*[@id="sidSELECT_MODEL_ACTION"]').click()

    #move to form
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('frame')
    # model typing
    driver.find_element_by_id('sidIN_NAME').send_keys(model)
    # search button
    driver.find_element_by_id('sidSEARCH_BUTTON').click()

    while True:
        try:
            driver.title
            sleep(5)
            print('Please Select Model')
        except:
            break

    # move back to main
    driver.switch_to.window(driver.window_handles[0])
    #select date installed
    driver.find_element_by_xpath('//*[@id="sidDISP_INSTALL_DATE"]').send_keys(date_installed)
    #select customer
    Select(driver.find_element_by_xpath('//*[@id="sidSTATUS_NAME"]')).select_by_index(1)
    #warranty date
    driver.find_element_by_xpath('//*[@id="sidDISP_WTY_START_DATE"]').send_keys(date_installed)
    wty_end = dt.strptime(date_installed,'%Y-%m-%d')+relativedelta(years=1)
    wty_end = wty_end.strftime('%Y-%m-%d')
    wty_end
    
    driver.find_element_by_xpath('//*[@id="sidDISP_WTY_END_DATE"]').send_keys(wty_end)
    sleep(0.2)
    #add equipment
    driver.find_element_by_xpath('//*[@id="sidADD_BUTTON_IMAGE"]').click()
    #return
    driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()

In [ ]:
# customer tab
driver.find_element_by_xpath('//*[@id="ACC_MGT_LINK"]').click()


In [ ]:
customer_code = 'FMSV00079'
#clear
driver.find_element_by_xpath('//*[@id="sidCLEAR_BUTTON_IMAGE"]').click()

driver.find_element_by_xpath('//*[@id="sidIN_ACC_CODE"]').send_keys(customer_code)
#search
driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()
# click first row
driver.find_element_by_xpath('//*[@id="g[0].sidDISP_CODE_data"]').click()

In [ ]:
date_installed = str(input('Date Installed: ' )) #'2023-06-30'


In [ ]:
date_installed = str(input('Date Installed: ' )) #'2023-06-30'
install_list = '''


'''
install_list = install_list.strip().split('\n')
for install in install_list:
    sn = install.strip().split('	')[1]
    model = install.strip().split('	')[0]
    print(model,sn)
    create_eqp(model,sn,date_installed)
print('Done')

In [ ]:
# equipment data
sn = '6V609K008'
model = 'VP-3500HD'
create_eqp(model,sn,date_installed)
print(f'Done for {model}: {sn}')

In [ ]:
driver.switch_to.window(driver.window_handles[0])

In [ ]:
#create EQP
driver.find_element_by_xpath('//*[@id="sidADD_BUTTON_IMAGE_2"]').click()
#search sn

driver.find_element_by_xpath('//*[@id="sidBODY_NO"]').send_keys(sn)
driver.find_element_by_xpath('//*[@id="sidBODY_NO"]').send_keys(Keys.RETURN)
#click model
driver.find_element_by_xpath('//*[@id="sidSELECT_MODEL_ACTION"]').click()

#move to form
# driver.switch_to.window(driver.window_handles[1])
# driver.switch_to.frame('frame')
# # model typing
# driver.find_element_by_id('sidIN_NAME').send_keys(model)
# # search button
# driver.find_element_by_id('sidSEARCH_BUTTON').click()

### Ship out HMU

In [ ]:
# driver.find_element_by_xpath('')

In [ ]:
# basic page first row
driver.find_element_by_xpath('//*[@id="search_result[0].sidRS_REPAIR_ID_data"]').click()

In [ ]:

for i in range(47,50):
    # status empty
    Select(driver.find_element_by_xpath('//*[@id="sidHOLD_CODE"]')).select_by_index(0) #clear QA
    Select(driver.find_element_by_xpath('//*[@id="sidTEMPORARY_STATUS_CODE"]')).select_by_index(1) #hold suppend
    #save
    driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
    #authorization
    driver.find_element_by_xpath('//*[@id="sidRPR_STATUS_BUTTON_5"]').click()
    # Cancel
    Select(driver.find_element_by_xpath('//*[@id="sidRPR_COMP_TYPE_CODE"]')).select_by_index(5)
    #tender lost
    driver.find_element_by_xpath('//*[@id="sidAUTH_NOTE_DECLINE"]').send_keys('Tender lost')
    # complete
    driver.find_element_by_xpath('//*[@id="sidAUTHORIZATION_COMPLETE_BUTTON_IMAGE"]').click()
    #return
#     driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()
    
    # ship out
    driver.find_element_by_xpath('//*[@id="sidRPR_STATUS_BUTTON_10"]').click()
    # complete
    driver.find_element_by_xpath('//*[@id="sidCOMPLETE_BUTTON_IMAGE"]').click()
    # next button
    driver.find_element_by_xpath('//*[@id="sidSHOW_NEXT_REPAIR"]').click()

In [ ]:
# ship out
driver.find_element_by_xpath('//*[@id="sidRPR_STATUS_BUTTON_10"]').click()
# complete
driver.find_element_by_xpath('//*[@id="sidCOMPLETE_BUTTON_IMAGE"]').click()
# next button
driver.find_element_by_xpath('//*[@id="sidSHOW_NEXT_REPAIR"]').click()

### Close job

In [ ]:
# clear button
driver.find_element_by_xpath('//*[@id="sidCLEAR_BUTTON_IMAGE"]').click()
# Status shipped
Select(driver.find_element_by_xpath('//*[@id="sidIN_REPAIR_STATUS"]')).select_by_index(9)
# Search
driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()
ttl = driver.find_element_by_xpath('//*[@id="TH_search_result"]/thead/tr[1]/th/div/span').get_attribute('innerHTML')
ttl = int(ttl.split('/')[1][:-1])
print(ttl)

In [ ]:

for i in range(ttl):
    # first row
    driver.find_element_by_xpath('//*[@id="search_result[0].sidRS_REPAIR_ID_data"]').click()
    # close button
    driver.find_element_by_xpath('//*[@id="sidRPR_STATUS_BUTTON_11"]').click()
    try:
        auto_accept(driver)
        print(i+1,'ROHS failed')
    except:
        print(i+1,'ROHS passed')
    driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()
print('Done')

### Add date to Installation, WTY


In [ ]:
# first row
driver.find_element_by_xpath('//*[@id="g[0].sidDISP_BODY_NO_data"]').click()

In [ ]:
# copy wty start
ins_date = driver.find_element_by_xpath('//*[@id="sidDISP_WTY_START_DATE"]').get_attribute('value')
#install date
driver.find_element_by_xpath('//*[@id="sidDISP_INSTALL_DATE"]').send_keys(ins_date)


In [ ]:
# save
driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
sleep(0.3)
# return
driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()

In [ ]:
while True:
    # first row
    driver.find_element_by_xpath('//*[@id="g[0].sidDISP_BODY_NO_data"]').click()
    # copy wty start
    ins_date = driver.find_element_by_xpath('//*[@id="sidDISP_WTY_START_DATE"]').get_attribute('value')
    #install date
    driver.find_element_by_xpath('//*[@id="sidDISP_INSTALL_DATE"]').send_keys(ins_date)
    sleep(0.3)
    # save
    driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
    # sleep(0.3)
    # return
    driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()
    ans = str(input('Continue?([Y]/N) ') or 'Y')
    if ans=='Y':
        print('Continue',ans)
    else:
        
        print('Break')
        break

In [ ]:
while True:
    # first row
    driver.find_element_by_xpath('//*[@id="g[0].sidDISP_BODY_NO_data"]').click()
    # copy wty start
    ins_date = driver.find_element_by_xpath('//*[@id="sidDISP_WTY_START_DATE"]').get_attribute('value')
    #install date
    if ins_date == '':
        date_sold = driver.find_element_by_xpath('//*[@id="sidDISP_SOLD_DATE"]').get_attribute('value')
        ins_date = date_sold
        if date_sold == '':
            create_date = driver.find_element_by_xpath('//*[@id="sidCREATE_TIME"]').get_attribute('innerHTML')
            create_date = create_date.split('&')[0]
            
            ins_date = dt.strptime(create_date,'%Y-%m-%d')+relativedelta(years=-1)
            ins_date = ins_date.strftime('%Y-%m-%d')
            
    driver.find_element_by_xpath('//*[@id="sidDISP_INSTALL_DATE"]').send_keys(ins_date)
    sleep(0.3)
    # save
    driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
    sleep(1)
    # return
    driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()
    ans = str(input('Continue?([Y]/N) ') or 'Y')
    if ans=='Y':
        print('Continue',ans)
    else:
        
        print('Break')
        break

In [ ]:
date_sold = driver.find_element_by_xpath('//*[@id="sidDISP_SOLD_DATE"]').get_attribute('value')

In [ ]:
date_sold != ''

In [ ]:
create_date = driver.find_element_by_xpath('//*[@id="sidCREATE_TIME"]').get_attribute('innerHTML')
create_date = create_date.split('&')[0]
create_date

In [ ]:
ins_date = dt.strptime(create_date,'%Y-%m-%d')+relativedelta(years=-1)
ins_date.strftime('%Y-%m-%d')

### Add Other ID

In [ ]:
id_list ='''

'''
id_list = id_list.strip().split('\n')
for rma_id in id_list:
    other_id = rma_id.strip().split('	')[1]
    rma = rma_id.strip().split('	')[0]
    print(rma,other_id)
    
    
    driver.find_element_by_xpath('//*[@id="sidIN_REPAIR_ID"]').clear()
    driver.find_element_by_xpath('//*[@id="sidIN_REPAIR_ID"]').send_keys(rma)
    driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()
    oid = driver.find_element_by_xpath('//*[@id="search_result[0].sidRS_SC_ID"]').get_attribute('innerHTML')
    if oid =='':
        #basic page
        driver.find_element_by_xpath('//*[@id="search_result[0].sidRS_REPAIR_ID_data"]').click()
        #Other ID
        driver.find_element_by_xpath('//*[@id="sidSC_ID"]').send_keys(other_id)
    #     save
        driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
        sleep(0.2)
        #return
        driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()
    else: print(rma,'Other ID not empty')

In [ ]:
len(id_list)

### Edit Equipments


In [99]:
# move back to main
driver.switch_to.window(driver.window_handles[0])
#equipment tab
driver.find_element_by_xpath('//*[@id="ACC_MGT_LINK"]').click()
#clear
driver.find_element_by_xpath('//*[@id="sidCLEAR_BUTTON_IMAGE"]').click()

In [112]:
customer_code = 'ANP00024'
# customer_code = 'FMSV00303'
# move back to main
driver.switch_to.window(driver.window_handles[0])
#equipment tab
driver.find_element_by_xpath('//*[@id="ACC_MGT_LINK"]').click()
#clear
driver.find_element_by_xpath('//*[@id="sidCLEAR_BUTTON_IMAGE"]').click()
driver.find_element_by_xpath('//*[@id="sidIN_ACC_CODE"]').send_keys(customer_code)
driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()
# first row
driver.find_element_by_xpath('//*[@id="g[0].sidDISP_CODE_data"]').click()


In [113]:
while True:
    try:
        #first equipments
        driver.find_element_by_xpath('//*[@id="equipments[0].sidEQP_BODY_NO_data"]').click()
        sleep(0.2)
        #change customer
        driver.find_element_by_xpath('//*[@id="sidSELECT_ACCOUNT_ACTION"]').click()

        #move to form
        driver.switch_to.window(driver.window_handles[-1])

        while True:
            try:
                driver.title
                sleep(1)
#                 print(driver.title)
            except:
                print('Next action!!!')
                break


        # move back to main
        driver.switch_to.window(driver.window_handles[0])
        #save
        driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
        #back
        driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()
        sleep(0.2)
    except Exception as e:
        print(e)
        break

Next action!!!
Next action!!!
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="equipments[0].sidEQP_BODY_NO_data"]"}
  (Session info: chrome=120.0.6099.71)
Stacktrace:
	GetHandleVerifier [0x00007FF7F8BF82B2+55298]
	(No symbol) [0x00007FF7F8B65E02]
	(No symbol) [0x00007FF7F8A205AB]
	(No symbol) [0x00007FF7F8A6175C]
	(No symbol) [0x00007FF7F8A618DC]
	(No symbol) [0x00007FF7F8A9CBC7]
	(No symbol) [0x00007FF7F8A820EF]
	(No symbol) [0x00007FF7F8A9AAA4]
	(No symbol) [0x00007FF7F8A81E83]
	(No symbol) [0x00007FF7F8A5670A]
	(No symbol) [0x00007FF7F8A57964]
	GetHandleVerifier [0x00007FF7F8F70AAB+3694587]
	GetHandleVerifier [0x00007FF7F8FC728E+4048862]
	GetHandleVerifier [0x00007FF7F8FBF173+4015811]
	GetHandleVerifier [0x00007FF7F8C947D6+695590]
	(No symbol) [0x00007FF7F8B70CE8]
	(No symbol) [0x00007FF7F8B6CF34]
	(No symbol) [0x00007FF7F8B6D062]
	(No symbol) [0x00007FF7F8B5D3A3]
	BaseThreadInitThunk [0x00007FFA2F2D7344+20]
	RtlUserThreadStart [0x00007FFA2

In [90]:
# move back to main
driver.switch_to.window(driver.window_handles[0])
#save
driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
#back
driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()

In [52]:
driver.switch_to.window(driver.window_handles[0])

In [87]:
for handle in driver.window_handles:
    driver.switch_to.window(handle)
    print(driver.title)

ExFM:Equipment Info.
ExFM:Customer Search


In [69]:
driver.switch_to.window(driver.window_handles[0])

In [66]:
driver.title

'ExFM:Customer Search'

In [ ]:
# driver.find_element_by_xpath('')

In [ ]:
# driver.find_element_by_xpath('')

In [ ]:
# driver.find_element_by_xpath('')

In [ ]:
# driver.find_element_by_xpath('')

In [ ]:
# driver.find_element_by_xpath('')

In [ ]:
# driver.find_element_by_xpath('')